# 02. Limpieza de datos

Este notebook contiene la limpieza del dataset de víctimas del conflicto armado en Colombia.

## Importación de Librerías

Importamos las librerías necesarias para el análisis de datos.

In [14]:
import pandas as pd
import csv
import os

## Carga del Dataset

Cargamos el dataset de víctimas del conflicto armado desde el archivo CSV.

In [15]:
# Define the relative path to the data file
data_path = '../data/raw/victimas_por_hechos_departamental_20250416.csv'

# Load the dataset
try:
    # Attempt to read with UTF-8 encoding first
    df = pd.read_csv(data_path, encoding='utf-8') 
except UnicodeDecodeError:
    try:
        # Fallback to Latin-1 encoding if UTF-8 fails
        df = pd.read_csv(data_path, encoding='latin1') 
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        df = None # Indicate failure by setting df to None

# Verificar si el dataset se cargó correctamente
if df is not None:
    print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
else:
    print("Error al cargar el dataset. Por favor, verifica la ruta y la codificación del archivo.")

Dataset cargado exitosamente. Dimensiones: (2431164, 17)


# Eliminacion de columnas innecesarias
Se eliminaran las columnas que no sean necesarias para el analisis como NOM_RPT, COD_PAI y PAIS

In [16]:
# Mostrar forma inicial del dataframe y vista previa
print(f"Forma inicial del dataframe: {df.shape}")
print(f"Columnas del dataframe: {df.columns.tolist()}")

Forma inicial del dataframe: (2431164, 17)
Columnas del dataframe: ['FECHA_CORTE', 'NOM_RPT', 'COD_PAIS', 'PAIS', 'COD_ESTADO_DEPTO', 'ESTADO_DEPTO', 'PARAM_HECHO', 'HECHO', 'SEXO', 'ETNIA', 'DISCAPACIDAD', 'CICLO_VITAL', 'PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']


In [17]:
# Columnas que se eliminarán
columns_to_drop = ["NOM_RPT", "COD_PAIS", "PAIS", "FECHA_CORTE"]

# Eliminar columnas especificadas
df = df.drop(columns=columns_to_drop)

In [18]:
# Mostrar forma final del dataframe
print(f"Forma final del dataframe después de eliminar columnas: {df.shape}")
print(f"Columnas del dataframe: {df.columns.tolist()}")
df.head()

Forma final del dataframe después de eliminar columnas: (2431164, 13)
Columnas del dataframe: ['COD_ESTADO_DEPTO', 'ESTADO_DEPTO', 'PARAM_HECHO', 'HECHO', 'SEXO', 'ETNIA', 'DISCAPACIDAD', 'CICLO_VITAL', 'PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']


,COD_ESTADO_DEPTO,ESTADO_DEPTO,PARAM_HECHO,HECHO,SEXO,ETNIA,DISCAPACIDAD,CICLO_VITAL,PER_OCU,PER_DECLA,PER_UBIC,PER_SA,EVENTOS
0,13,Bolivar,5,Desplazamiento forzado,Hombre,Gitano (RROM) (Acreditado RA),Ninguna,entre 18 y 28,3.0,4.0,3.0,3.0,3.0
1,20,Cesar,5,Desplazamiento forzado,Mujer,Raizal del Archipielago de San Andres y Provid...,Multiple,entre 61 y 100,1.0,NaN,NaN,NaN,1.0
2,18,Caqueta,5,Desplazamiento forzado,Hombre,Gitano (RROM) (Acreditado RA),Ninguna,entre 29 y 60,2.0,NaN,NaN,NaN,2.0
3,95,Guaviare,5,Desplazamiento forzado,Hombre,Negro(a) o Afrocolombiano(a),Fisica,entre 61 y 100,11.0,7.0,4.0,4.0,12.0
4,18,Caqueta,2,Amenaza,Hombre,Indigena (Acreditado RA),Por Establecer,entre 12 y 17,1.0,1.0,1.0,1.0,1.0


# Corrección de errores de codificación y unificación de categorías

Se identificaron y reemplazaron valores con errores de codificación en varias columnas, con el fin de mejorar la calidad y consistencia de los datos. A continuación, se detallan los ajustes realizados:

### Columna: `ESTADO_DEPTO`

Valores corregidos:

* `"Nari�o"`
* `"NariÃ±o"`

Reemplazados por:

* `"Nariño"`

---

### Columna: `HECHO`

Valores corregidos:

* `"Desaparici�n forzada"`
* `"DesapariciÃ³n forzada"`

Reemplazados por:

* `"Desaparición forzada"`

---

* `"VinculaciÃ³n de NiÃ±os NiÃ±as y Adolescentes a Actividades Relacionadas con grupos armados"`
* `"Vinculaci�n de Ni�os Ni�as y Adolescentes a Actividades Relacionadas con grupos armados"`

Reemplazados por:

* `"Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados"`

---

* `"Minas Antipersonal, Munici�n sin Explotar y Artefacto Explosivo improvisado"`

Reemplazado por:

* `"Minas Antipersonal, Munición sin Explotar y Artefacto Explosivo improvisado"`

---

### Columna: `ETNIA`

Unificación de categorías para facilitar el análisis:

Valores originales:

* `"Negro (Acreditado RA)"`
* `"Afrocolombiano (Acreditado RA)"`

Reemplazados por:

* `"Negro(a) o Afrocolombiano(a) (Acreditado RA)"`


In [19]:
# Corrección de errores de codificación

df['ESTADO_DEPTO'] = df['ESTADO_DEPTO'].replace({
    'Nari�o': 'Nariño',
    'NariÃ±o': 'Nariño'
})

df['HECHO'] = df['HECHO'].replace({
    'Desaparici�n forzada': 'Desaparición forzada',
    'DesapariciÃ³n forzada': 'Desaparición forzada',
    'VinculaciÃ³n de NiÃ±os NiÃ±as y Adolescentes a Actividades Relacionadas con grupos armados':
        'Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados',
    'Vinculaci�n de Ni�os Ni�as y Adolescentes a Actividades Relacionadas con grupos armados':
        'Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados',
    'Minas Antipersonal, Munici�n sin Explotar y Artefacto Explosivo improvisado':
        'Minas Antipersonal, Munición sin Explotar y Artefacto Explosivo improvisado'
})

df['ETNIA'] = df['ETNIA'].replace({
    'Negro (Acreditado RA)': 'Negro(a) o Afrocolombiano(a) (Acreditado RA)',
    'Afrocolombiano (Acreditado RA)': 'Negro(a) o Afrocolombiano(a) (Acreditado RA)'
})

df['CICLO_VITAL'] = df['CICLO_VITAL'].replace({
'entre 61 y 100': 'entre 60 y 110'
})


# Limpieza y Validación de Grupos Únicos para Reemplazo de NaN por Cero

Este análisis tiene como objetivo **justificar y ejecutar** el reemplazo de valores faltantes (`NaN`) por `0` en las variables de conteo:

- `PER_OCU`
- `PER_DECLA`
- `PER_UBIC`
- `PER_SA`
- `EVENTOS`

El procedimiento se realiza **únicamente si se confirma que no existen registros duplicados** por grupo definido a partir de:

- `FECHA_CORTE`
- `HECHO`
- `ESTADO_DEPTO`
- `SEXO`
- `CICLO_VITAL`
- `ETNIA`

De esta forma se garantiza que cada fila representa un grupo único y que los valores nulos pueden interpretarse como **"cero personas/eventos registrados"**.



In [20]:
# Paso 1: Definir columnas
columnas_grupo = ['COD_ESTADO_DEPTO','ESTADO_DEPTO', 'PARAM_HECHO','HECHO', 'SEXO',  'ETNIA', 'DISCAPACIDAD', 'CICLO_VITAL']
variables_conteo = ['PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']
# Paso 2: Agrupar y sumar valores de conteo
df = df.groupby(columnas_grupo, as_index=False)[variables_conteo].sum()

In [21]:
# Paso 3: Reemplazar NaN por 0
df[variables_conteo] = df[variables_conteo].fillna(0)

In [22]:
# Paso 4: Validar que no queden NaN
print("¿Quedan NaN después del reemplazo?")
print(df[variables_conteo].isna().sum())

¿Quedan NaN después del reemplazo?
PER_OCU      0
PER_DECLA    0
PER_UBIC     0
PER_SA       0
EVENTOS      0
dtype: int64


# Generacion de COD_RPT de victimas
En esta seccion se genera el COD_RPT de las victimas para su posterior uso en la generacion de las primary keys.

In [23]:
# Asegurarse de que el índice sea limpio, secuencial y comience desde 0
df = df.reset_index(drop=True)

# Crear columna 'COD_RPT' como ID incremental comenzando desde 1
df['COD_RPT'] = df.index + 1

# Reordenar columnas para que 'COD_RPT' esté al inicio
df = df[['COD_RPT'] + [col for col in df.columns if col != 'COD_RPT']]


# Muestra del dataset de víctimas limpio 

In [26]:
df.head(10)

,COD_RPT,COD_ESTADO_DEPTO,ESTADO_DEPTO,PARAM_HECHO,HECHO,SEXO,ETNIA,DISCAPACIDAD,CICLO_VITAL,PER_OCU,PER_DECLA,PER_UBIC,PER_SA,EVENTOS
0,1,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano (RROM) (Acreditado RA),Ninguna,entre 18 y 28,0.0,22.0,0.0,0.0,0.0
1,2,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano (RROM) (Acreditado RA),Ninguna,entre 29 y 59,0.0,0.0,7.0,7.0,0.0
2,3,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano (RROM) (Acreditado RA),Ninguna,entre 29 y 60,0.0,0.0,8.0,8.0,0.0
3,4,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano(a) ROM,Fisica,entre 29 y 59,0.0,7.0,14.0,14.0,0.0
4,5,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano(a) ROM,Fisica,entre 29 y 60,0.0,34.0,11.0,11.0,0.0
5,6,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano(a) ROM,Ninguna,entre 18 y 28,0.0,0.0,8.0,8.0,0.0
6,7,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano(a) ROM,Ninguna,entre 29 y 59,0.0,21.0,7.0,0.0,0.0
7,8,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Gitano(a) ROM,Ninguna,entre 29 y 60,0.0,102.0,34.0,2.0,0.0
8,9,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Indigena,Auditiva,entre 29 y 59,0.0,14.0,0.0,0.0,0.0
9,10,0,SIN DEFINIR,1,Acto terrorista / Atentados / Combates / Enfre...,Hombre,Indigena,Auditiva,entre 29 y 60,0.0,44.0,0.0,0.0,0.0


# Exportamos los datos limpios a un archivo CSV
Se exporta el archivo CSV con los datos limpios en en ´/data/processed/victimas_por_hechos_departamental_20250416.csv´ para su posterior análisis

In [69]:
df.to_csv(
    path_or_buf='../data/processed/victimas_por_hechos_departamental_20250416.csv',
    sep=',',
    na_rep='',
    header=True,
    index=False,
    encoding='utf-8',
    quoting=csv.QUOTE_MINIMAL,
    lineterminator=os.linesep,
    quotechar='"',
    decimal='.',
    errors='strict'
)